<a href="https://githubtocolab.com/neurallatents/nlb_tools/blob/main/examples/tutorials/gpfa_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPFA Demo

While `basic_example.ipynb` used a smoothing implementation to generate rate predictions for the benchmark, this notebook will run GPFA, a better modeling method, using the Python package [`elephant`](https://github.com/NeuralEnsemble/elephant), which should produce far better results. We recommend first viewing `basic_example.ipynb` for more explanation of the `nlb_tools` functions we use here.

## 1. Setup

Below, we import the necessary functions from `nlb_tools` and additional standard packages. Note that you will need to install `elephant`, which should install with it `neo`, and `quantities` if you don't already have them. Additionally, you'll need `scikit-learn>=0.23` for the Poisson GLM used in this notebook.

In [1]:
## Install packages if necessary
# !pip install elephant
# !pip install -U scikit-learn
# !pip install git+https://github.com/neurallatents/nlb_tools.git

In [2]:
## Imports

from nlb_tools.nwb_interface import NWBDataset
from nlb_tools.make_tensors import make_train_input_tensors, make_eval_input_tensors, make_eval_target_tensors, save_to_h5
from nlb_tools.evaluation import evaluate

import numpy as np
import pandas as pd
import h5py
import neo
import quantities as pq
from elephant.gpfa import GPFA
from sklearn.linear_model import PoissonRegressor, Ridge

In [3]:
## If necessary, download dataset from DANDI
# !pip install dandi
# !dandi install https://dandiarchive.org/dandiset/000138 # replace URL with URL for dataset you want

## 2. Loading data

Below, we enter the name of the dataset, the path to the dataset files, as well as a prefix to filter out specific files, in order to load the data.

In [4]:
## Load dataset

dataset_name = 'mc_maze_large'
datapath = '~/lvm/code/dandi/000138/sub-Jenkins/'
prefix = f'*ses-large'
dataset = NWBDataset(datapath, prefix)

FileNotFoundError: Specified file or directory not found

## 3. Input prep

`elephant`'s implementation of GPFA takes its input in the form of lists of `neo.SpikeTrain`s. Here, we'll use `make_train_input_tensor` or `make_eval_input_tensor` to extract the data we want to model before converting it into the desired format.

In [ ]:
## Dataset preparation

# Choose the phase here, either 'val' or 'test'
phase = 'val'

# Choose bin width and resample
bin_width = 5
dataset.resample(bin_width)

# Create suffix for group naming later
suffix = '' if (bin_width == 5) else f'_{int(round(bin_width))}'

In [ ]:
## Make train input data

# Generate input tensors
train_trial_split = 'train' if (phase == 'val') else ['train', 'val']
train_dict = make_train_input_tensors(dataset, dataset_name=dataset_name, trial_split=train_trial_split, save_file=False)

# Unpack input data
train_spikes_heldin = train_dict['train_spikes_heldin']
train_spikes_heldout = train_dict['train_spikes_heldout']

In [ ]:
## Make eval input data

# Generate input tensors
eval_split = phase
eval_dict = make_eval_input_tensors(dataset, dataset_name=dataset_name, trial_split=eval_split, save_file=False)

# Unpack data
eval_spikes_heldin = eval_dict['eval_spikes_heldin']

In [ ]:
## Convert spiking array to SpikeTrains

def array_to_spiketrains(array):
    """Converts trial x time x channel spiking arrays to list of list of neo.SpikeTrain"""
    stList = []
    # Loop through trials
    for trial in range(len(array)):
        trialList = []
        # Loop through channels
        for channel in range(array.shape[2]):
            # Get spike times and counts
            times = np.where(array[trial, :, channel])[0]
            counts = array[trial, times, channel].astype(int)
            train = np.repeat(times, counts)
            # Create neo.SpikeTrain
            st = neo.SpikeTrain(times*bin_width*pq.ms, t_stop=array.shape[1]*bin_width*pq.ms)
            trialList.append(st)
        stList.append(trialList)
    return stList

# Run conversion
train_st_heldin = array_to_spiketrains(train_spikes_heldin)
eval_st_heldin = array_to_spiketrains(eval_spikes_heldin)

## 4. Running GPFA

Now that we have properly formatted data, we'll run GPFA. This step may take quite a while, depending on your machine and the chosen parameters.

In [ ]:
## Run GPFA

# Set parameters
bin_size = bin_width * pq.ms
latent_dim = 10

# Train GPFA on train data and apply on test data
gpfa = GPFA(bin_size=bin_size, x_dim=latent_dim)
train_factors = gpfa.fit_transform(train_st_heldin)
eval_factors = gpfa.transform(eval_st_heldin)

# Extract and reshape factors to 3d array
train_factors = np.stack([train_factors[i].T for i in range(len(train_factors))])
eval_factors = np.stack([eval_factors[i].T for i in range(len(eval_factors))])


Initializing parameters using factor analysis...

Fitting GPFA model...


## 5. Generating rate predictions

Now that we have our latent factors at the specified resolution, we can map these factors to the spiking data.

In [ ]:
## Basic data prep

# Get input arrays
train_spikes_heldin = train_dict['train_spikes_heldin']
train_spikes_heldout = train_dict['train_spikes_heldout']

# Assign variables
tlength = train_spikes_heldin.shape[1]
numtrain = train_spikes_heldin.shape[0]
numeval = eval_spikes_heldin.shape[0]
numheldin = train_spikes_heldin.shape[2]
numheldout = train_spikes_heldout.shape[2]

# Reshape data to 2d for regression
flatten3d = lambda x: x.reshape(-1, x.shape[2])
train_spikes_heldin_s = flatten3d(train_spikes_heldin)
train_spikes_heldout_s = flatten3d(train_spikes_heldout)
train_factors_s = flatten3d(train_factors)
eval_factors_s = flatten3d(eval_factors)

In [ ]:
## Define fitting functions

def fit_rectlin(train_input, eval_input, train_output, alpha=0.0):
    # Fit linear regression
    lr = Ridge(alpha=alpha)
    lr.fit(train_input, train_output)
    train_pred = lr.predict(train_input)
    eval_pred = lr.predict(eval_input)
    # Rectify to prevent negative or 0 rate predictions
    train_pred[train_pred < 1e-10] = 1e-10
    eval_pred[eval_pred < 1e-10] = 1e-10
    return train_pred, eval_pred

def fit_poisson(train_input, eval_input, train_output, alpha=0.0):
    train_pred = []
    eval_pred = []
    # train Poisson GLM for each output column
    for chan in range(train_output.shape[1]):
        pr = PoissonRegressor(alpha=alpha, max_iter=500)
        pr.fit(train_input, train_output[:, chan])
        train_pred.append(pr.predict(train_input))
        eval_pred.append(pr.predict(eval_input))
    train_pred = np.vstack(train_pred).T
    eval_pred = np.vstack(eval_pred).T
    return train_pred, eval_pred

In [ ]:
## Make rate predictions

# fit GLMs for rate predictions
train_rates_heldin_s, eval_rates_heldin_s = fit_rectlin(train_factors_s, eval_factors_s, train_spikes_heldin_s)
train_rates_heldout_s, eval_rates_heldout_s = fit_poisson(train_rates_heldin_s, eval_rates_heldin_s, train_spikes_heldout_s)

# reshape output back to 3d
train_rates_heldin = train_rates_heldin_s.reshape((numtrain, tlength, numheldin))
train_rates_heldout = train_rates_heldout_s.reshape((numtrain, tlength, numheldout))
eval_rates_heldin = eval_rates_heldin_s.reshape((numeval, tlength, numheldin))
eval_rates_heldout = eval_rates_heldout_s.reshape((numeval, tlength, numheldout))

## 6. Making the submission

Now, we'll make the submission dict manually. As described in `basic_example.ipynb`, you can also use the function `save_to_h5` from `make_tensors.py` to save the output as an h5 file for submission on EvalAI.

In [ ]:
## Prepare submission data

output_dict = {
    dataset_name + suffix: {
        'train_rates_heldin': train_rates_heldin,
        'train_rates_heldout': train_rates_heldout,
        'eval_rates_heldin': eval_rates_heldin,
        'eval_rates_heldout': eval_rates_heldout
    }
}

# To save as an h5 file:
# save_to_h5(output_dict, 'submission.h5')

## 7. Evaluation

Finally, we will create the target data with `make_eval_target_tensors` and evaluate our model if we ran on the 'val' phase. If the notebook was run on the 'test' phase, you would need to submit to the EvalAI challenge to get results.

In [ ]:
## Make data to test predictions with and evaluate

if phase == 'val':
    target_dict = make_eval_target_tensors(dataset, dataset_name=dataset_name, train_trial_split='train', eval_trial_split='val', include_psth=('mc_rtt' not in dataset_name), save_file=False)

    print(evaluate(target_dict, output_dict))

[{'area2_bump_split': {'coNLL': 3.438209105335957, 'vel R2': 0.5704200272213656, 'psth R2': 0.476563983978655}}]


## Summary

In this notebook, we used `nlb_tools` and `elephant` to run GPFA on a dataset for the Neural Latents Benchmark.